## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


In [1]:
#load libraries
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.image as mpimg

## Camera Calibration

In [2]:
# Camera Calibration

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgshape = gray.shape[::-1]# getting shape

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        
        
# Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, imgshape, None, None)


## Tools

In [3]:
# pipeline for image preparation
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Stack each channel
    # color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
    # Combine thresholds
    combined = np.zeros_like(sxbinary)
    combined[(sxbinary == 1) | (s_binary == 1) ] = 1

    return combined

# Distortion Correction
def undistort(img, mtx, dist):
    # 0) Defining Points
    img_size = (img.shape[1], img.shape[0])
    offset = 400
    #src = np.float32([[595,450],[690,450],[1053,678],[260,678]])
    src = np.float32([[595,450],[690,450],[1135,720],[219,720]])
    dst = np.float32([[offset, 0], [img_size[0]-offset, 0], [img_size[0]-offset, img_size[1]], [offset, img_size[1]]])   
    # 1) Undistort using mtx and dist
    undist = cv2.undistort(img, mtx, dist, None, mtx)    
    # 2) Calculate M
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    # 3) Unwarp
    warped = cv2.warpPerspective(undist, M, img_size)
    
    return warped, M, M_inv

# find pixels in windows
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

# initial fit 
def fit_initial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    
    
    left_fitx, right_fitx, ploty, left_fit, right_fit, left_fit_real, right_fit_real = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)    

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

#     # Plots the left and right polynomials on the lane lines
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')

    return out_img, left_fit, right_fit

# fit poly
def fit_poly(img_shape, leftx, lefty, rightx, righty):
     ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/465 # meters per pixel in x dimension 
    left_fit_real = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_real = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    return left_fitx, right_fitx, ploty, left_fit, right_fit, left_fit_real, right_fit_real

# search for new pixels
def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 50

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty, left_fit, right_fit, left_fit_real, right_fit_real = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result, left_fitx, right_fitx, ploty, left_fit, right_fit, left_fit_real, right_fit_real

# measure curvature
def measure_curvature(ploty, left_fit_real, right_fit_real):
        
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ym_per_pix = 30/720 # meters per pixel in y dimension
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_real[0]*y_eval*ym_per_pix + left_fit_real[1])**2)**1.5) / np.absolute(2*left_fit_real[0])
    right_curverad = ((1 + (2*right_fit_real[0]*y_eval*ym_per_pix + right_fit_real[1])**2)**1.5) / np.absolute(2*right_fit_real[0])
    
    return left_curverad, right_curverad

# measure lane position
def lanepos(ploty, left_fit_real,right_fit_real) :
    
    y_eval = np.max(ploty) 
    
    xm_per_pix = 3.7/465 # meters per pixel in x dimension 
    ym_per_pix = 30/720
     
    center_car = 640 * xm_per_pix
    
    y_eval = y_eval * ym_per_pix
    
    # position right road
    position_right = right_fit_real[0]* y_eval**2 + right_fit_real[1]* y_eval + right_fit_real[2]
    position_left = left_fit_real[0]* y_eval**2 + left_fit_real[1]* y_eval + left_fit_real[2]
    
    street_width = position_right - position_left
        
    caroff = center_car - position_left - street_width/2 

    return caroff

# class for the lines
class Line():
    def __init__(self):
        self.left_fit = []
        self.right_fit = []
        self.initial_fit_counter=0
        
class ProcessImage:
    def __init__(self, line):
        self.line = line
    def __call__(self, image):
        image = process_image(image, line)
        return image

# plot image
def map_down(image, line):
    
    # check predictions
    left_fit = line.left_fit
    right_fit = line.right_fit
    
    # Read the image
    img_raw = image
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # create binary image with image processing pipeline
    binary = pipeline(img_raw)
    
    # change perspective, warp
    binary_warped, M, Minv = undistort(binary, mtx, dist)
    
    # find line initially or if lost
    #if 1==1:
    if line.left_fit==[]:
        # fit polynomial with window slices
        out_img, left_fit , right_fit = fit_initial(binary_warped)
        # count these fits
        line.initial_fit_counter=line.initial_fit_counter+1

    # Search around founded poly 
    result, left_fitx, right_fitx, ploty, left_fit_new, right_fit_new, left_fit_real, right_fit_real = search_around_poly(binary_warped, left_fit, right_fit)
    
    line.left_fit = left_fit_new;
    line.right_fit = right_fit_new;
    
    # Calculate curve
    left_curverad, right_curverad = measure_curvature(ploty, left_fit_real, right_fit_real)
    curvature = 0.5*(left_curverad + right_curverad)

    # Calculate lane 
    caroff = lanepos(ploty, left_fit_real,right_fit_real)
    caroff = round(caroff,2)
    
    # Write some Text
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    pos1 = (10,30)
    pos2 = (10,60)
    pos3 = (10,90)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2

    #cv2.putText(img_raw,'initial fit:'+ str(line.initial_fit_counter), pos3, font, fontScale,fontColor,lineType)
    cv2.putText(img_raw,'curvature in m:'+ str(int(curvature)), pos1, font, fontScale,fontColor,lineType)
    cv2.putText(img_raw,'lane position in m:'+ str(caroff), pos2, font, fontScale,fontColor,lineType)
    
    

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img_raw, 1, newwarp, 0.3, 0)
    
    return result   

# class for video processing
def process_image(image, line): 
    result = map_down(image, line);
    return result


## Test Tools

In [4]:

# Test tools and create imaages for documentation

fig_size_x = 20
fig_size_y = 15
text_size  = 15

waittime = 0

###################
# Test Calibration)
img = cv2.imread('camera_cal/calibration1.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

undistimg = cv2.undistort(img, mtx, dist, None, mtx)

#plot

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(fig_size_x, fig_size_y))
f.tight_layout()
ax1.imshow(img, cmap='gray')
ax1.set_title('Original Image', fontsize=text_size)
ax2.imshow(undistimg , cmap='gray')
ax2.set_title('Corrected Image', fontsize=text_size)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# save image
f.savefig('output_images/camera_calibration.png')

###################
# Test Distortion Correction
img = cv2.imread('test_images/straight_lines2.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

undistimg = cv2.undistort(img, mtx, dist, None, mtx)

#plot

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(fig_size_x, fig_size_y))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=text_size)
ax2.imshow(undistimg)
ax2.set_title('Corrected Image', fontsize=text_size)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# save image
f.savefig('output_images/distortion_corrected.png')


##################
#Test pipeline

#plot
img_raw = cv2.imread('test_images/straight_lines2.jpg')
img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
img_pipeline = pipeline(img_raw)
warped, M, Minv = undistort(img_pipeline, mtx, dist)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(fig_size_x, fig_size_y))
f.tight_layout()
ax1.imshow(img_raw)
ax1.set_title('Original Image', fontsize=text_size)
ax2.imshow(img_pipeline, cmap='gray')
ax2.set_title('Image after pipeline', fontsize=text_size)
ax3.imshow(warped, cmap='gray')
ax3.set_title('Warped Image', fontsize=text_size)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# save image
f.savefig('output_images/pipeline_warp.png')

##################
#Test lane line pixel

#plot
img = warped

out_img, left_fit, right_fit = fit_initial(warped)

result, left_fitx, right_fitx, ploty, left_fit, right_fit, left_fit_real, right_fit_real = search_around_poly(warped, left_fit, right_fit)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(fig_size_x, fig_size_y))
f.tight_layout()
ax1.imshow(img, cmap='gray')
ax1.set_title('Original Image', fontsize=text_size)
ax2.imshow(out_img, cmap='gray')
ax2.set_title('Initial Detection', fontsize=text_size)
ax3.imshow(result)
ax3.set_title('Area Search', fontsize=text_size)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# save image
f.savefig('output_images/line_detection.png')

##################
#Test map_down

#plot
img_raw = cv2.imread('test_images/straight_lines2.jpg')
img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)

testline = Line()

img = map_down(img_raw, testline);

f, (ax1) = plt.subplots(1, 1, figsize=(fig_size_x, fig_size_y))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=text_size)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# save image
f.savefig('output_images/map_down.png')


cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## Run code on video

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
white_output = 'test_video.mp4'

clip1 = VideoFileClip("project_video.mp4")
#clip1 = VideoFileClip("challenge_video.mp4").subclip(0,5)
#clip1 = VideoFileClip("harder_challenge_video.mp4").subclip(1)

line = Line()
white_clip = clip1.fl_image(ProcessImage(line)) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|          | 3/1260 [00:00<00:48, 25.72it/s, now=None]

Moviepy - Building video test_video.mp4.
Moviepy - Writing video test_video.mp4



Moviepy - Done !
Moviepy - video ready test_video.mp4
CPU times: user 2min 21s, sys: 29.5 s, total: 2min 50s
Wall time: 52.9 s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))